# Getting Started with Azure Quantum Resource Estimation using Q# and Python

👋 Welcome to the Azure Quantum Resource Estimator. In this notebook we will
guide you how to estimate and analyze the physical resource estimates of a
quantum program targeted for execution based on the architecture design of a
fault-tolerant quantum computer. As a running example we are using a multiplier.

## Setup

Let's connect to the Azure Quantum workspace and select the Azure Quantum
Resource Estimator as target.  Examples for other targets include QPUs to
execute quantum programs on today's quantum computers, or simulators to simulate
the functional behavior of a quantum program.  You can find examples for such
targets in other notebooks in the _Sample Gallery_. We are also importing the
`Microsoft.Quantum.Numerics` package that we will require for our example
algorithm.

In [ ]:
import qsharp.azure
targets = qsharp.azure.connect(
   resourceId="",
   location="")

In [ ]:
qsharp.packages.add("Microsoft.Quantum.Numerics")
qsharp.azure.target("microsoft.estimator")

## Implementing the algorithm

As running example algorithm we are creating a multiplier using the [MultiplyI](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.arithmetic.multiplyi) operation.  We can configure the size of the multiplier with a bitwidth parameter. The operation will have two input registers with that bitwidth, and one output register with the size of twice the bitwidth.

In [ ]:
EstimateMultiplication: any = None # Make Python recognize the Q# function (optional)

In [ ]:
%%qsharp

open Microsoft.Quantum.Arithmetic;

operation EstimateMultiplication(bitwidth : Int) : Unit {
    use factor1 = Qubit[bitwidth];
    use factor2 = Qubit[bitwidth];
    use product = Qubit[2 * bitwidth];
    
    MultiplyI(LittleEndian(factor1), LittleEndian(factor2), LittleEndian(product));
}

## Estimating the algorithm

Let's now estimate the physical resources for this operation using a bitwidth of 8 and the default assumptions.  We can submit the operation to the resource estimation target using the `qsharp.azure.execute` function.

In [ ]:
result = qsharp.azure.execute(EstimateMultiplication, bitwidth=8)

The simplest way to inspect the results of the job is to output them to the notebook. This will output a table with the overall physical resource counts. You can further inspect more details about the resource estimates by collapsing various groups which have more information. For example, if you collapse the *Logical qubit parameters* group, you can see that the quantum error correction (QEC) code distance is 13. In the last group you can see the physical qubit properties that were assumed for this estimation. For example, we see that the time to perform a single-qubit measurement and a single-qubit gate are assumed to be 100 ns and 50 ns, respectively.

In [ ]:
result

If you prefer a more compact version of the table, in which the descriptions are
provided by means of tooltips, you can write:

In [ ]:
result.summary

We can also programmatically access all the values that can be passed to the job execution and see which default values were assumed:

In [ ]:
result['jobParams']

We see that there are three input parameters that can be customized: `qubitParams`, `qecScheme`, and `errorBudget`.

### Qubit parameters

The first parameter `qubitParams` is used to specify qubit parameters.  When
modeling the physical qubit abstractions, we distinguish between two different
physical instruction sets that are used to operate the qubits.  The physical
instruction set can be either *gate-based* or *Majorana*.  A gate-based
instruction set provides single-qubit measurement, single-qubit gates (incl. T
 gates), and two-qubit gates.  A Majorana instruction set provides a physical T
 gate, single-qubit measurement and two-qubit joint measurement operations.

Qubit parameters can be completely customized.  Before we show this, we show hot
to choose from six pre-defined qubit parameters, four of which have gate-based
instruction sets and two with a Majorana instruction set.  An overview of all
pre-defined qubit parameters is provided by the following table:

| Pre-defined qubit parameters | Instruction set | References                                                                                                 |
|------------------------------|-----------------|------------------------------------------------------------------------------------------------------------|
| `"qubit_gate_ns_e3"`         | gate-based      | [arXiv:2003.00024](https://arxiv.org/abs/2003.00024), [arXiv:2111.11937](https://arxiv.org/abs/2111.11937) |
| `"qubit_gate_ns_e4"`         | gate-based      | [arXiv:2003.00024](https://arxiv.org/abs/2003.00024), [arXiv:2111.11937](https://arxiv.org/abs/2111.11937) |
| `"qubit_gate_us_e3"`         | gate-based      | [arXiv:1701.04195](https://arxiv.org/abs/1701.04195)                                                       |
| `"qubit_gate_us_e4"`         | gate-based      | [arXiv:1701.04195](https://arxiv.org/abs/1701.04195)                                                       |
| `"qubit_maj_ns_e4"`          | Majorana        | [arXiv:1610.05289](https://arxiv.org/abs/1610.05289)                                                       |
| `"qubit_maj_ns_e6"`          | Majorana        | [arXiv:1610.05289](https://arxiv.org/abs/1610.05289)                                                       |

Pre-defined qubit parameters can be selected by specifying the `name` field in
the `qubitParams`.  If no value is provided, `"qubit_gate_ns_e3"` is chosen as
the default qubit parameters.

Let's re-run resource estimation for our running example on the Majorana-based
qubit parameters `"qubit_maj_ns_e6"`.

In [ ]:
result = qsharp.azure.execute(EstimateMultiplication,
            bitwidth=8,
            jobParams={
                "qubitParams": {
                    "name": "qubit_maj_ns_e6"
                }})
result

Let's inspect the physical counts programmatically. For example, we can show all physical resource estimates and their breakdown using the `physicalCounts` field in the result data. This will show the logical qubit error and logical T-state error rates required to match the error budget. By default runtimes are shown in nanoseconds.

In [ ]:
result['physicalCounts']

We can also explore details about the T factory that was created to execute this algorithm.

In [ ]:
result['tfactory']

Next, we are using this data to produce some explanations of how the T factories produce the required T states.

In [ ]:
tfactory = result["tfactory"]
breakdown = result["physicalCounts"]["breakdown"]
producedTstates = breakdown["numTfactories"] * breakdown["numTfactoryRuns"] * tfactory["numTstates"]

print(f"""A single T factory produces {tfactory["numTstates"]} T state(s) with an error rate of {tfactory["logicalErrorRate"]:.2e} (required T state error rate is {breakdown["requiredLogicalTstateErrorRate"]:.2e}).""")
print(f"""{breakdown["numTfactories"]} copie(s) of a T factory are executed {breakdown["numTfactoryRuns"]} time(s) to produce {producedTstates} T states ({breakdown["numTstates"]} are required by the algorithm).""")
print(f"""A single T factory is composed of {tfactory["numRounds"]} rounds of distillation:""")
for round in range(tfactory["numRounds"]):
    print(f"""- {tfactory["numUnitsPerRound"][round]} {tfactory["unitNamePerRound"][round]} unit(s)""")

Custom qubit parameters must completely specify all required parameters.  These are the values that are
considered when the `instructionSet` is `"GateBased"`.

| Field (*required)               | Description                                                          |
|---------------------------------|----------------------------------------------------------------------|
| `name`                          | Some descriptive name for the parameters                             |
| `oneQubitMeasurementTime`*      | Operation time for single-qubit measurement ($t_{\rm meas}$) in ns   |
| `oneQubitGateTime`*             | Operation time for single-qubit Clifford gate ($t_{\rm gate}$) in ns |
| `twoQubitGateTime`              | Operation time for two-qubit Clifford gate in ns                     |
| `tGateTime`                     | Operation time for single-qubit non-Clifford gate in ns              |
| `oneQubitMeasurementErrorRate`* | Error rate for single-qubit measurement                              |
| `oneQubitGateErrorRate`*        | Error rate for single-qubit Clifford gate ($p$)                      |
| `twoQubitGateErrorRate`         | Error rate for two-qubit Clifford gate                               |
| `tGateErrorRate`                | Error rate to prepare single-qubit non-Clifford state ($p_T$)        |

The values for `twoQubitGateTime` and `tGateTime` default to `oneQubitGateTime`
when not specified; the values for `twoQubitGateErrorRate` and `tGateErrorRate`
default to `oneQubitGateErrorRate` when not specified.

A minimum template for qubit parameters based on a gate-based instruction set
with all required values is:

```json
{
    "qubitParams": {
        "instructionSet": "GateBased",
        "oneQubitMeasurementTime": <time string>,
        "oneQubitGateTime": <time string>,
        "oneQubitMeasurementErrorRate": <double>,
        "oneQubitGateErrorRate": <double>
    }
}
```

For time units, you need to specify time strings which are double-precision
floating point numbers followed by a space and a unit prefix which is `ns`, `µs`
(alternatively `us`), `ms`, or `s`.

These are the values that are considered when the `instructionSet` is
`"Majorana"`.

| Field (*required)                   | Description                                                         |
|-------------------------------------|---------------------------------------------------------------------|
| `name`                              | Some descriptive name for the parameters                            |
| `oneQubitMeasurementTime`*          | Operation time for single-qubit measurement ($t_{\rm meas}$) in ns  |
| `twoQubitJointMeasurementTime`      | Operation time for two-qubit joint measurement in ns                |
| `tGateTime`                         | Operation time for single-qubit non-Clifford gate in ns             |
| `oneQubitMeasurementErrorRate`*     | Error rate for single-qubit measurement                             |
| `twoQubitJointMeasurementErrorRate` | Error rate for two-qubit joint measurement                          |
| `tGateErrorRate`*                   | Error rate to prepare single-qubit non-Clifford state ($p_T$)       |

The values for `twoQubitJointMeasurementTime` and `tGateTime` default to
`oneQubitGateTime` when not specified; the value for
`twoQubitJointMeasurementErrorRate` defaults to `oneQubitMeasurementErrorRate`
when not specified.

A minimum template for qubit parameters based on a Majorana instruction set with
all required values is:

```json
{
    "qubitParams": {
        "instructionSet": "Majorana",
        "oneQubitMeasurementTime": <time string>,
        "oneQubitMeasurementErrorRate": <double>,
        "tGateErrorRate": <double>
    }
}
```

### QEC schemes

To execute practical-scale quantum applications, we require operations with very
low error rates. These error rate targets are typically beyond the capabilities
of raw physical qubits. To overcome this limitation, quantum error correction
(QEC) and fault-tolerant computation are two crucial techniques that form the
building blocks of large-scale quantum computers. First, QEC allows us to
compose multiple error-prone physical qubits and build a more reliable logical
qubit that preserves quantum information better than the underlying physical
qubits. Several QEC schemes have been developed since the last three decades,
including popular schemes such as the Shor-code, surface code, color codes and
others, and recently, the [Hastings-Haah
code](https://arxiv.org/abs/2107.02194). These schemes vary based on the number
of physical qubits they require, the connectivity among qubits and other
factors. By using QEC techniques, we can achieve a fault-tolerant quantum
computation, enabling reliable storing and processing of quantum information in
the presence of noise. To store information reliably, we require that the QEC
scheme is able to suppress errors when the physical qubits meet a certain
threshold error rate. To process information, we require fault-tolerant
operations that allow applications to perform general purpose quantum
computations efficiently and limit the spread of errors that occur while
computing with logical qubits. Schemes for fault-tolerant operations include
techniques such as lattice surgery and transversal operations. Together, QEC and
fault-tolerance techniques bridges the accuracy gap between quantum hardware and
algorithms.

The error correction code distance (or just code distance in short) is a
parameter that controls the number of errors that can be corrected, and thus the
error rate of the logical qubits and the number of physical qubits required to
encode them.  The higher the code distance, the better the accuracy, but also
the higher the amount of physical qubits.  The goal is to find the minimum code
distance that can achieve the required error rate set for a particular
application.  We will explain later in this notebook how a global error budget
is provided as input and how it is distributed throughout the estimation,
including the logical error rate of logical qubits.

We follow the standard way of modeling logical error rates using an exponential
model parameterized by the code distance $d$, physical error rate $p$, QEC
threshold $p^*$.  The physical error rate $p$ is extracted from the qubit
parameters above as the worst-case error rate any physical Clifford operation in
the device.  In particular, we set $p = {}$ `max(oneQubitMeasurementErrorRate,
oneQubitGateErrorRate, twoQubitGateErrorRate)` for qubit parameters with a
gate-based instruction set, and $p = {}$ `max(oneQubitMeasurementErrorRate,
twoQubitJointMeasurementErrorRate)` for qubit parameters with a Majorana
instruction set.  QEC schemes typically have a error rate threshold $p^*$ below
which error correction suppresses errors.

Our current implementation uses the formula

$$
P = a\left(\frac{p}{p^*}\right)^{\frac{d+1}{2}}
$$

as the generic model.  The exact parameters for each pre-defined QEC scheme
(including  a crossing pre-factor $a$ which can be extracted numerically for
simulations) are listed below.

In Azure Quantum Resource Estimation we can abstract the quantum error
correction scheme based on the above formula by providing values for the
crossing pre-factor $a$ and the error correction threshold $p^*$.  Further, one
needs to specify the logical cycle time, i.e., the time to execute a single
logical operation, which depends on the code distance and the  physical
operation time assumptions of the underlying physical qubits.  Finally, a second
formula computes the number of physical qubits required to encode one logical
qubit based on the code distance.

As with the physical qubit parameters, one can choose from several pre-defined
QEC schemes, can extend pre-defined ones, and can provide custom schemes by
providing all parameters.  Note that QEC schemes are tightly connected to the
physical instruction set of the physical qubit parameters, and therefore are
defined specifically for one of the two instruction sets.

We provide three pre-defined QEC schemes, two `"surface_code"` protocols for
gate-based and Majorana physical instruction sets, and the `"floquet_code"`
protocol that is so far only implemented for a Majorana physical instruction set
in the resource estimator.

| QEC scheme     | Instruction set | References                                                                                                 |
|----------------|-----------------|------------------------------------------------------------------------------------------------------------|
| `surface_code` | gate-based      | [arXiv:1208.0928](https://arxiv.org/abs/1208.0928), [arXiv:1009.3686](https://arxiv.org/abs/1009.3686)     |
| `surface_code` | Majorana        | [arXiv:1909.03002](https://arxiv.org/abs/1909.03002), [arXiv:2007.00307](https://arxiv.org/abs/2007.00307) |
| `floquet_code` | Majorana        | [arXiv:2202.11829](https://arxiv.org/abs/2202.11829)                                                       |

In case of `"surface_code"` the corresponding scheme is selected based on the
qubit type of the physical qubit parameters.  The gate-based surface code is
based on [[arXiv:1208.0928](https://arxiv.org/abs/1208.0928)] and
[[arXiv:1009.3686](https://arxiv.org/abs/1009.3686)]. The surface code for
Majorana qubits is based on
[[arXiv:1909.03002](https://arxiv.org/abs/1909.03002)] and
[[arXiv:2007.00307](https://arxiv.org/abs/2007.00307)] (replacing 8 steps to
measure a single stabilizer in the former reference by 20 steps to measure all
stabilizers). The floquet code, which can only be selected for Majorana qubits,
is based on [[arXiv:2202.11829](https://arxiv.org/abs/2202.11829)].

Pre-defined qubit parameters can be selected by specifying the `name` field in
the `qecScheme` parameter.  If no value is provided, `"surface_code"` is used as
default value.

Let's re-run resource estimation for our running example on the Majorana-based
qubit parameters with a Floquet code.

In [ ]:
result_maj_floquet = qsharp.azure.execute(EstimateMultiplication,
            bitwidth=8,
            jobParams={
                "qubitParams": {
                    "name": "qubit_maj_ns_e6"
                },
                "qecScheme": {
                    "name": "floquet_code"
                }})
result_maj_floquet

To specify a QEC scheme the user has to specify 2 values, the
`errorCorrectionThreshold` and the `crossingPrefactor`, as well as 2 formulas
for the `logicalCycleTime`, and the `physicalQubitsPerLogicalQubit`.  A template
for QEC schemes is as follows:

```json
{
    "qecScheme": {
        "crossingPrefactor": <double>,
        "errorCorrectionThreshold": <double>,
        "logicalCycleTime": <formula string>,
        "physicalQubitsPerLogicalQubit": <formula string>
    }
}
```

Inside the formulas, the user can make use of the following variables

* `oneQubitGateTime`
* `twoQubitGateTime`
* `oneQubitMeasurementTime`
* `twoQubitJointMeasurementTime`

whose value is taken from the corresponding field from the physical qubit
parameters (note that some variables are not available based on the qubit
parameters' instruction set), as well as the variable

* `codeDistance`

for the code distance computed for the logical qubit, based on the physical
qubit properties, the error correction threshold, and the crossing prefactor.
The time variables and `codeDistance` can be used to describe the
`logicalCycleTime` formula.  For the formula `physicalQubitsPerLogicalQubit`
only the `codeDistance` can be used.

### Error budget

The third parameter `errorBudget` models the total error budget $\epsilon$.  It
sets the overall allowed error for the algorithm, i.e., the number of times it
is allowed to fail.  Its value must be between 0 and 1 and the default value is
0.001, which corresponds to 0.1%, and means that the algorithm is allowed to
fail once in 1000 executions.  This parameter is highly application specific.
For example, if one is running Shor's algorithm for factoring integers, a large
value for the error budget may be tolerated as one can check that the output are
indeed the prime factors of the input.  On the other hand, a much smaller error
budget may be needed for an algorithm solving a problem with a solution which
cannot be efficiently verified.  This budget

$$
  \epsilon = \epsilon_{\log} + \epsilon_{\rm dis} + \epsilon_{\rm syn}
$$

is uniformly distributed and applies to errors $\epsilon_{\log}$ to implement
logical qubits, an error budget $\epsilon_{\rm dis}$ to produce T states through
distillation, and an error budget $\epsilon_{\rm syn}$ to synthesize rotation
gates with arbitrary angles.  Note that for distillation and rotation synthesis,
the respective error budgets $\epsilon_{\rm dis}$ and $\epsilon_{\rm syn}$ are
uniformly distributed among all required T states and all required rotation
gates, respectively. If there are no rotation gates in the input algorithm, the
error budget is uniformly distributed to logical errors and T state errors.

Next, we re-run the last experiment with a an error budget of 10%.

In [ ]:
result_maj_floquet_e1 = qsharp.azure.execute(EstimateMultiplication,
            bitwidth=8,
            jobParams={
                "qubitParams": {
                    "name": "qubit_maj_ns_e6"
                },
                "qecScheme": {
                    "name": "floquet_code"
                },
                "errorBudget": 0.1})
result_maj_floquet_e1

## Next steps

We hope you enjoyed this notebook and found it helpful in exploring the physical resource estimates for quantum programs. Here are some suggested next steps:

* Try estimating the resources for a different Q# program.
* Explore how qubit parameters and QEC schemes affect the error correction code distance of the logical qubit.
* Use the output data to derive logical qubit properties.
* Learn how to setup complex resource estimation experiments in the _Advanced analysis of estimates_ notebook.